# Bonus 1

# 0. import

In [164]:
import requests
import scrapy
import lxml
from lxml import etree
import json
from bs4 import BeautifulSoup
import sys
import xlrd
from xlwt import *
import pandas as pd
import urllib

# 1. get operator set

In [185]:
with open('well_data.json') as fp:
    welldata = json.load(fp)

In [186]:
welldata

[{'District': '6',
  'Formatted API': '001-00001',
  'Operator Name': 'E & B Natural Resources Management Corporation',
  'Operator Code': 'E0100',
  'Field Name': 'Livermore',
  'Field Code': '404',
  'API': '00100001',
  'Lease Name': 'Greenville Investment Group',
  'Well': '1',
  'Well Status': 'A',
  'Pool WellTypes': 'OG',
  'Section': '7',
  'Township': '03S',
  'Range': '03E',
  'Base Meridian': 'MD',
  'Area Code': '00',
  'Area Name': 'Any Area',
  'Latitude': 37.693671528,
  'Longitude': -121.689627066,
  'GISSourceCode': 'gps',
  'DatumCode': '83',
  'BLMWell': 'N',
  'DryHole': 'N',
  'Directional': 'N',
  'Hydraulically Fractured': ' ',
  'SPUD Date': '12/30/1966',
  'Completion Date': '',
  'Abandoned Date': ''},
 {'District': '6',
  'Formatted API': '001-00002',
  'Operator Name': 'Chevron U.S.A. Inc.',
  'Operator Code': 'C5640',
  'Field Name': 'Hospital Nose Gas (ABD)',
  'Field Code': '314',
  'API': '00100002',
  'Lease Name': 'Hancock-Signal (NCT-1) Wente',
  'Wel

# 2. get data from well data based on operator name 

In [187]:
operater_list = []
for i in range(0,len(welldata)):
    operater_list.append(welldata[i]['Operator Name'])
operater_set = set(operater_list)   

In [190]:
def get_one_opt(opt_name):
    well = []
    api = []
    for i in welldata:
        if i['Operator Name'] == opt_name:
            well.append(i)
            opt_api = i['API']
            api.append(opt_api)        
    return well,api
            
    

## -example of how to use function def get_one_opt(opt_name)
### we use 'Dos Rios, Inc.' as operator name


In [191]:
b = get_one_opt('Dos Rios, Inc.')[0]
df_well = pd.DataFrame(b)
api_list= get_one_opt('Dos Rios, Inc.')[1]

In [192]:
df_well

,API,Abandoned Date,Area Code,Area Name,BLMWell,Base Meridian,Completion Date,DatumCode,Directional,District,...,Longitude,Operator Code,Operator Name,Pool WellTypes,Range,SPUD Date,Section,Township,Well,Well Status
0,01120098,,00,Any Area,N,MD,,83,N,6,...,-122.026615,D3530,"Dos Rios, Inc.",DG,02W,07/07/1976,1,17N,1,I
1,01120151,,00,Any Area,N,MD,,83,N,6,...,-122.022354,D3530,"Dos Rios, Inc.",DG,01W,06/24/1980,31,18N,1-31,I
2,01120189,,00,Any Area,N,MD,,83,N,6,...,-122.020307,D3530,"Dos Rios, Inc.",DG,01W,02/14/1981,6,17N,1-6,I
3,01120624,,00,Any Area,N,MD,06/30/1998,83,N,6,...,-122.000260,D3530,"Dos Rios, Inc.",DG,01W,06/05/1998,8,15N,1-8,A
4,01924230,11/07/2012,00,Any Area,Y,MD,03/03/2003,83,N,5,...,-120.540746,D3530,"Dos Rios, Inc.",OG,13E,01/07/2003,22,16S,5-22,P
5,10120801,,00,Any Area,N,MD,09/26/2008,83,N,6,...,-121.701242,D3530,"Dos Rios, Inc.",DG,02E,09/03/2008,24,14N,3-24,I
6,10120802,,00,Any Area,N,MD,11/06/2008,83,N,6,...,-121.716328,D3530,"Dos Rios, Inc.",DG,02E,09/28/2008,23,14N,1-23,A
7,10120804,,00,Any Area,N,MD,11/12/2008,83,N,6,...,-121.689414,D3530,"Dos Rios, Inc.",DG,03E,10/02/2008,19,14N,1-19,A
8,10120397,,00,Any Area,N,MD,06/13/1990,83,N,6,...,-121.708820,D3530,"Dos Rios, Inc.",DG,02E,05/22/1990,23,14N,1-23,I


# 3. get production data based on operator name

## -example of how to get all production data with operator name as 'Dos Rios, Inc.'

In [165]:
#run this if you don't download all the production file before!!!!!!!
def get_one_production(api_key):
    url = 'https://secure.conservation.ca.gov/WellSearch/Download/ToExcelProductionData?APInumber=%s'%(api_key)
    urllib.request.urlretrieve(url, 'production_{}.xlsx'.format(api_key))

def get_all_production(api_list):
    for i in api_list:
        get_one_production(i)
        
get_all_production(api_list)

In [197]:
prod_list = []
for j in api_list:
    tem = pd.read_excel('production_{}.xlsx'.format(j),header=3)
    try:
        exec('df_prod%s=tem'%(j))
        prod_list.append(tem)
    except:
        continue
    

prod_list
df_prod_all = pd.concat(prod_list)
df_prod_final = df_prod_all.set_index(['API Number','Production Date'])

In [198]:
df_prod_final

Oil Produced (bbl)  Water Produced (bbl)  \
API Number Production Date                                             
1120098    2018 Total                      0.0                   0.0   
           Feb-2018                        0.0                   0.0   
           Jan-2018                        0.0                   0.0   
           2017 Total                      0.0                   0.0   
           Dec-2017                        0.0                   0.0   
           Nov-2017                        0.0                   0.0   
           Oct-2017                        0.0                   0.0   
           Sep-2017                        0.0                   0.0   
           Aug-2017                        0.0                   0.0   
           Jul-2017                        0.0                   0.0   
           Jun-2017                        0.0                   0.0   
           May-2017                        0.0                   0.0   
           Apr-2017                        0.0                   0.0   
           Mar-2017                        0.0                   0.0   
           Feb-2017                        0.0                   0.0   
           Jan-2017                        0.0                   0.0   
           2016 Total                      0.0                   0.0   
           Dec-2016                        0.0                   0.0   
           Nov-2016                        0.0                   0.0   
           Oct-2016                        0.0                   0.0   
           Sep-2016                        0.0                   0.0   
           Aug-2016                        0.0                   0.0   
           Jul-2016                        0.0                   0.0   
           Jun-2016                        0.0                   0.0   
           May-2016                        0.0                   0.0   
           Apr-2016                        0.0                   0.0   
           Mar-2016                        0.0                   0.0   
           Feb-2016                        0.0                   0.0   
           Jan-2016                        0.0                   0.0   
           2015 Total                      0.0                   0.0   
...                                        ...                   ...   
10120397   Aug-1992                        0.0                 501.0   
           Jul-1992                        0.0                 454.0   
           Jun-1992                        0.0                 433.0   
           May-1992                        0.0                 432.0   
           Apr-1992                        0.0                 380.0   
           Mar-1992                        0.0                 428.0   
           Feb-1992                        0.0                 387.0   
           Jan-1992                        0.0                 389.0   
           1991 Total                      0.0                4450.0   
           Dec-1991                        0.0                 466.0   
           Nov-1991                        0.0                 427.0   
           Oct-1991                        0.0                 418.0   
           Sep-1991                        0.0                 408.0   
           Aug-1991                        0.0                 392.0   
           Jul-1991                        0.0                 422.0   
           Jun-1991                        0.0                 396.0   
           May-1991                        0.0                 407.0   
           Apr-1991                        0.0                 375.0   
           Mar-1991                        0.0                 274.0   
           Feb-1991                        0.0                 270.0   
           Jan-1991                        0.0                 195.0   
           1990 Total                      0.0                 517.0   
           Dec-1990                        0.0                 190.0

# 4. how to get well and production data for all operator 

## we can use this code below:

In [ ]:
for i in operater_set

## and apply these two example before to get all the data:)